<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=fe7ea92cd654c54c81b35b00001ebd91ae3d7c6c50ded53f552f1fa66e7edb35
  Stored in directory: c:\users\tanishq anand\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [18]:
import folium
import wget
import pandas as pd

In [19]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

from folium.plugins import MarkerCluster

Explanation: This line imports the MarkerCluster plugin from the folium.plugins module. Folium is a Python library used for creating interactive <br>maps. The MarkerCluster plugin allows you to group multiple markers on a map into clusters to improve performance and user experience when dealing <br>with a large number of markers.

from folium.plugins import MousePosition

Explanation: This imports the MousePosition plugin from folium.plugins. The MousePosition plugin adds a widget to the map that displays the <br>geographic coordinates (latitude and longitude) of the mouse cursor as it moves over the map.
<br>
from folium.features import DivIcon

Explanation: This line imports the DivIcon class from folium.features. The DivIcon plugin allows you to create custom HTML icons that can be added <br>to markers on the map. These icons can contain custom HTML content such as text or images.<br>


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [20]:
# Download and read the `spacex_launch_geo.csv`
# spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv("F:/Imp courses/IBM DATA Science/10 Applied Data Science Capstone/Interactive visual analytics & Dashboard/Labs/spacex_launch_geo.csv")

In [21]:
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [22]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df.head(5)


,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

Step 1: Selecting Sub-columns: This line selects specific columns from the DataFrame spacex_df and assigns the result back to spacex_df. The <br>selected columns are 'Launch Site', 'Lat' (Latitude), 'Long' (Longitude), and 'class'. It assumes that spacex_df contains these columns, likely <br>representing data related to SpaceX launches.

Group by 'Launch Site' and select the first entry (assuming it's the first launch at each site)

launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

Step 2: Grouping by 'Launch Site': Here, spacex_df.groupby(['Launch Site'], as_index=False) groups the DataFrame spacex_df by the 'Launch Site'

 column. The parameter as_index=False ensures that 'Launch Site' remains a column rather than becoming the index of the resulting DataFrame.

Selecting First Entry: .first() is used after grouping to select the first entry (row) for each unique launch site. This assumes that the first<br> entry represents relevant information about each launch site, such as its geographical coordinates and classification ('class').



Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [23]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [26]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(50,50),
        icon_anchor=(0,0),
        html='<div style="font-size: 50; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Explaination 

Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name

circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

Create a Circle: This line creates a circle on the map using the folium.Circle class.<br>
nasa_coordinate: The coordinates for the NASA Johnson Space Center. This should be a tuple, e.g., (latitude, longitude).<br>
radius=1000: The radius of the circle in meters.<br>
color='#d35400': The color of the circle's outline, in this case, a specific shade of orange.<br>
fill=True: Fills the circle with the same color.<br>
add_child(folium.Popup('NASA Johnson Space Center')): Adds a popup to the circle that displays the text "NASA Johnson Space Center" when clicked.<br>

 Create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name

marker = folium.map.Marker(<br>
    nasa_coordinate,<br>
    # Create an icon as a text label<br>
    icon=DivIcon(<br>
        icon_size=(20,20),<br>
        icon_anchor=(0,0),<br>        )
    )

Create a Marker: This line creates a marker on the map using the folium.map.Marker class.<br>
nasa_coordinate: The same coordinates for the NASA Johnson Space Center.<br>
icon=DivIcon(...): Creates a custom icon for the marker using the DivIcon class.<br>
icon_size=(20,20): Sets the size of the icon.
icon_anchor=(0,0): Sets the anchor point of the icon. (0,0) means the top-left corner of the icon.<br>
html=' style="font-size: 12; color:#d35400;">% 'NASA JSC':<br>
 Sets the HTML content of the icon. This creates a div element with the text "NASA JSC" styled with a font size of 12 and the same orange color.<br>

site_map.add_child(circle)<br>
site_map.add_child(marker)<br>
Add Circle to Map: This line adds the previously created circle to the map object site_map.<br>
Add Marker to Map: This line adds the previously created marker to the map object site_map.<br>

Summary:<br>
Purpose: The code creates and adds a visual marker and a circle with a popup to a map at the NASA Johnson Space Center's location.

Steps:<br>
Create Circle: A circle is created at the specified coordinates with a radius of 1000 meters, colored orange, and a popup displaying "NASA Johnson Space Center".<br>
Create Marker: A marker is created at the same coordinates with a custom icon displaying "NASA JSC" in orange text.<br>
Add to Map: Both the circle and the marker are added to the map object site_map.<br>
<br>

Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [27]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [28]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [29]:
marker_cluster = MarkerCluster()

# Purpose of MarkerCluster
# The MarkerCluster class is used to manage and display a large number of markers on a map efficiently. When there are many markers, showing them 
# all individually can clutter the map and make it difficult to interpret. MarkerCluster helps by grouping nearby markers together into a single 
# cluster marker. As you zoom in on the map, the cluster expands to show the individual markers.



MarkerCluster: This refers to a class provided by the folium.plugins module, which is part of the folium library.<br>
marker_cluster = MarkerCluster(): This line creates an instance of the MarkerCluster class and assigns it to the variable marker_cluster.


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [30]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [33]:
# Add the Marker cluster to the site map
site_map.add_child(marker_cluster)

# site_map.add_child(marker_cluster): Adds the previously created marker_cluster to the site_map. This allows the map to display the clustered markers.


# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for site_lat, site_long, marker_color in zip(spacex_df['Lat'], spacex_df['Long'], spacex_df['marker_color']):
# Loops through the spacex_df DataFrame, retrieving latitude (site_lat), longitude (site_long), and marker color (marker_color) for each row. 
# The zip function allows you to iterate over multiple columns simultaneously.


    site_coordinate = [site_lat, site_long] #Combines the latitude and longitude into a list representing the coordinates of the site.

    marker = folium.map.Marker(
        site_coordinate,
        # Create an icon as a text label
        icon=folium.Icon(color='white', 
                         icon_color=marker_color)
    )
#  Creates a Marker object for the given coordinates. The icon parameter customizes the marker's icon. Here, the folium.Icon is used with a
#  white background (color='white') and the icon_color is set based on the marker_color value from the DataFrame. The marker_color could indicate 
# the success or failure of a launch (e.g., green for success and red for failure).


    marker.add_to(marker_cluster)
    # marker.add_to(marker_cluster): Adds the created marker to the marker_cluster. This means the marker will be part of the cluster displayed on the map.


site_map
# site_map: Outputs the site_map object, which will display the map with the clustered markers when run in a Jupyter notebook or similar environment.


# The purpose of the provided code is to create an interactive map that visualizes SpaceX launch sites and highlights the outcomes of these launches
#  using different colored markers. Initially, a marker cluster is added to the map to group nearby markers, enhancing readability and preventing 
# clutter. For each launch site in the dataset, a marker is created at the respective latitude and longitude coordinates. The marker's color
#  indicates the outcome of the launch, with different colors representing success and failure. This approach allows users to quickly discern 
# patterns and results across various launch sites. The interactive features of the map, such as zooming and clicking on markers for additional
#  information, make it a valuable educational and analytical tool. By visualizing the data in this manner, the map provides a clear and engaging
#  way to explore SpaceX's launch history and outcomes.

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [34]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
# This code adds a MousePosition plugin to a Folium map (site_map) to display the latitude and longitude coordinates of the mouse pointer as it
#  moves over the map. Here's a detailed explanation of each part of the code:



formatter = "function(num) {return L.Util.formatNum(num, 5);};"
# This JavaScript function formats numbers to 5 decimal places. It will be used to format the latitude and longitude values displayed on the map.



mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# position='topright': Positions the coordinate display at the top right of the map.
# separator=' Long: ': Sets the separator between latitude and longitude values.
# empty_string='NaN': Displays 'NaN' when the mouse is not over the map.
# lng_first=False: Specifies that latitude should be displayed before longitude.
# num_digits=20: Sets the maximum number of digits for the coordinates (though it's mostly influenced by the formatter).
# prefix='Lat:': Adds a prefix before the latitude value.
# lat_formatter=formatter: Uses the formatter function to format latitude values.
# lng_formatter=formatter: Uses the formatter function to format longitude values.


site_map.add_child(mouse_position) #This line adds the MousePosition plugin to the map (site_map), enabling the display of coordinates as the mouse moves over the map.

site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [35]:
# This code defines a function calculate_distance to compute the distance between two geographical points on the Earth's surface using the Haversine
#  formula. Here's a step-by-step explanation:



from math import sin, cos, sqrt, atan2, radians
#This line imports the necessary mathematical functions from the math module.


def calculate_distance(lat1, lon1, lat2, lon2):
# This line defines a function named calculate_distance that takes four parameters: lat1, lon1, lat2, and lon2. These represent the latitude and 
# longitude of two points on the Earth's surface.


    # approximate radius of earth in km
    R = 6373.0
    # This line defines the approximate radius of the Earth in kilometers (6373.0 km).


    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    # These lines convert the latitude and longitude values from degrees to radians because trigonometric functions in Python work with radians.


    dlon = lon2 - lon1
    dlat = lat2 - lat1
# These lines calculate the difference between the longitudes (dlon) and the latitudes (dlat).


    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # The variable a is calculated using the Haversine formula to account for the spherical shape of the Earth.
# c is the angular distance in radians.


    distance = R * c
    return distance

# The variable distance is calculated by multiplying the Earth's radius R by the angular distance c.
# The function returns the computed distance in kilometers.
# In summary, the calculate_distance function uses the Haversine formula to calculate the great-circle distance between two points given their latitude and longitude. This formula is accurate for short distances and moderately accurate for longer distances on the Earth's surface.

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [36]:
# distance_railway = calculate_distance(lat1, lon1, lat2, lon2)
railway_marker = [28.55752, -80.80155]
launch_coordinate = [28.57337, -80.64669]
distance_railway = calculate_distance(railway_marker[0], railway_marker[1], launch_coordinate[0], launch_coordinate[1])
distance_railway  # distance in km


15.230651245141603

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [37]:
# create and add a folium.Marker on your selected closest raiwaly point on the map
# show the distance to the launch site using the icon property 
marker = folium.map.Marker(
        railway_marker,
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(400, 400),
            icon_anchor=(0, 0),
            html='<div style="font-size:400; color:#0c10f2;"><b>%s</b></div>' % str(round(distance_railway, 2))+' km',
            )
    )
marker.add_to(site_map)

site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [38]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
folium.PolyLine([railway_marker, launch_coordinate], color='blue').add_to(site_map)
site_map



Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [39]:
# Create a marker with distance to a closest city, coastline, highway, etc.
# Draw a line between the marker to the launch site
city   = [28.61200, -80.80788]
coastline = [28.5858, -80.79952]
highway   = [28.5402, -80.85079]

city_distance = calculate_distance(city[0], city[1], launch_coordinate[0], launch_coordinate[1])
coastline_distance = calculate_distance(coastline[0], coastline[1], launch_coordinate[0], launch_coordinate[1])
highway_distance = calculate_distance(highway[0], highway[1], launch_coordinate[0], launch_coordinate[1])

colors = ['red','orange','green']
html_colors = ['#dc3545','#fd7e14','#198754']

for coordinate ,distance, color, html_color in zip([city, coastline, highway], [city_distance, coastline_distance, highway_distance], colors, html_colors):
    marker = folium.map.Marker(
            coordinate,
            # Create an icon as a text label
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:'+html_color+';"><b>%s</b></div>' % str(round(distance, 2)) + 'km',
                )
            )
    marker.add_to(site_map)
    folium.PolyLine([coordinate, launch_coordinate], color=color).add_to(site_map)
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.

Explanation:<br>
From the visual analysis of the launch site KSC LC-39A we can clearly see that it is:<br>
relative close to railway (15.23 km)<br>
relative close to highway (20.28 km)<br>
relative close to coastline (14.99 km)<br>
Also the launch site KSC LC-39A is relative close to its closest city Titusville (16.32 km).<br>
Failed rocket with its high speed can cover distances like 15-20 km in few seconds. It could be potentially dangerous to populated areas.<br>


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
